# Custom DataLoader for Imbalanced dataset

* In this notebook we will use the higly imbalanced Protein Homology Dataset from [KDD cup 2004](https://www.kdd.org/kdd-cup/view/kdd-cup-2004/Data)

```
* The first element of each line is a BLOCK ID that denotes to which native sequence this example belongs. There is a unique BLOCK ID for each native sequence. BLOCK IDs are integers running from 1 to 303 (one for each native sequence, i.e. for each query). BLOCK IDs were assigned before the blocks were split into the train and test sets, so they do not run consecutively in either file.
* The second element of each line is an EXAMPLE ID that uniquely describes the example. You will need this EXAMPLE ID and the BLOCK ID when you submit results.
* The third element is the class of the example. Proteins that are homologous to the native sequence are denoted by 1, non-homologous proteins (i.e. decoys) by 0. Test examples have a "?" in this position.
* All following elements are feature values. There are 74 feature values in each line. The features describe the match (e.g. the score of a sequence alignment) between the native protein sequence and the sequence that is tested for homology.
```

## Initial imports

In [1]:
import numpy as np
import pandas as pd
import torch
from torch.optim import SGD, lr_scheduler

from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.models import TabMlp, WideDeep
from pytorch_widedeep.dataloaders import DataLoaderImbalanced, DataLoaderDefault
from torchmetrics import F1Score as F1_torchmetrics
from torchmetrics import Accuracy as Accuracy_torchmetrics
from torchmetrics import Precision as Precision_torchmetrics
from torchmetrics import Recall as Recall_torchmetrics
from pytorch_widedeep.metrics import Accuracy, Recall, Precision, F1Score, R2Score
from pytorch_widedeep.initializers import XavierNormal
from pytorch_widedeep.datasets import load_bio_kdd04

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import time
import datetime

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

# increase displayed columns in jupyter notebook
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 300)

/Users/javierrodriguezzaurin/.pyenv/versions/3.8.12/envs/wd38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = load_bio_kdd04(as_frame=True)
# drop columns we won't need in this example
df.drop(columns=["EXAMPLE_ID", "BLOCK_ID"], inplace=True)

df_train, df_valid = train_test_split(
    df, test_size=0.2, stratify=df["target"], random_state=1
)
df_valid, df_test = train_test_split(
    df_valid, test_size=0.5, stratify=df_valid["target"], random_state=1
)

continuous_cols = df.drop(columns=["target"]).columns.values.tolist()

In [3]:
# deeptabular
tab_preprocessor = TabPreprocessor(continuous_cols=continuous_cols, scale=True)
X_tab_train = tab_preprocessor.fit_transform(df_train)
X_tab_valid = tab_preprocessor.transform(df_valid)
X_tab_test = tab_preprocessor.transform(df_test)

# target
y_train = df_train["target"].values
y_valid = df_valid["target"].values
y_test = df_test["target"].values

In [4]:
# Define the model
input_layer = len(tab_preprocessor.continuous_cols)
output_layer = 1
hidden_layers = np.linspace(
    input_layer * 2, output_layer, 5, endpoint=False, dtype=int
).tolist()

deeptabular = TabMlp(
    column_idx=tab_preprocessor.column_idx,
    continuous_cols=tab_preprocessor.continuous_cols,
    mlp_hidden_dims=hidden_layers,
)
model = WideDeep(deeptabular=deeptabular)
model

WideDeep(
  (deeptabular): Sequential(
    (0): TabMlp(
      (cat_and_cont_embed): DiffSizeCatAndContEmbeddings(
        (cont_norm): BatchNorm1d(74, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (tab_mlp): MLP(
        (mlp): Sequential(
          (dense_layer_0): Sequential(
            (0): Dropout(p=0.1, inplace=False)
            (1): Linear(in_features=74, out_features=148, bias=True)
            (2): ReLU(inplace=True)
          )
          (dense_layer_1): Sequential(
            (0): Dropout(p=0.1, inplace=False)
            (1): Linear(in_features=148, out_features=118, bias=True)
            (2): ReLU(inplace=True)
          )
          (dense_layer_2): Sequential(
            (0): Dropout(p=0.1, inplace=False)
            (1): Linear(in_features=118, out_features=89, bias=True)
            (2): ReLU(inplace=True)
          )
          (dense_layer_3): Sequential(
            (0): Dropout(p=0.1, inplace=False)
            (1): Linear(in_featu

In [5]:
# Metrics from pytorch-widedeep
accuracy = Accuracy(top_k=2)
precision = Precision(average=False)

# # Metrics from torchmetrics
# accuracy = Accuracy_torchmetrics(average=None, num_classes=1)
# precision = Precision_torchmetrics(average="micro", num_classes=1)

# Optimizers
deep_opt = SGD(model.deeptabular.parameters(), lr=0.1)

# LR Scheduler
deep_sch = lr_scheduler.StepLR(deep_opt, step_size=3)

trainer = Trainer(
    model,
    objective="binary",
    lr_schedulers={"deeptabular": deep_sch},
    initializers={"deeptabular": XavierNormal},
    optimizers={"deeptabular": deep_opt},
    metrics=[accuracy, precision],
    verbose=1,
)

In [6]:
start = time.time()
trainer.fit(
    X_train={"X_tab": X_tab_train, "target": y_train},
    X_val={"X_tab": X_tab_valid, "target": y_valid},
    n_epochs=1,
    batch_size=32,
    custom_dataloader=DataLoaderImbalanced,
    oversample_mul=5,
)
print(
    "Training time[s]: {}".format(
        datetime.timedelta(seconds=round(time.time() - start))
    )
)

pd.DataFrame(trainer.history)

df_pred = trainer.predict(X_tab=X_tab_test)
print(classification_report(df_test["target"].to_list(), df_pred))
print("Actual predicted values:\n{}".format(np.unique(df_pred, return_counts=True)))

valid: 100%|█████████████████████████████████| 456/456 [00:01<00:00, 282.95it/s, loss=0.0998, metrics={'acc': 0.9853, 'prec': [0.3569]}]


Training time[s]: 0:00:02


predict: 100%|███████████████████████████████████████████████████████████████████████████████████████| 456/456 [00:00<00:00, 566.27it/s]


              precision    recall  f1-score   support

           0       1.00      0.99      0.99     14446
           1       0.39      0.85      0.54       130

    accuracy                           0.99     14576
   macro avg       0.70      0.92      0.76     14576
weighted avg       0.99      0.99      0.99     14576

Actual predicted values:
(array([0, 1]), array([14295,   281]))
